In [ ]:

import numpy as np
from astropy.stats import bootstrap
from astropy.utils import NumpyRNGContext
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def findFrontLocation(fracH):
    elementLocation = 0
    i=0
    while elementLocation == 0:
        if (fracH[i]-.5) > 0 :
            elementLocation=i
        i = i+1     
    return elementLocation  


def getFileDataArrays(i,j,k,scatter_path1,temperature_path):
    data = []
    data_i = []
    data_position = []
    data_direction = []
    data_polarization = []
    data_frequency = []
    data_freqoff = []
    data_lastpos = []
    data_vi=[]
    data_vf=[]
    
    
    with open(scatter_path1) as f:
        lines = f.readlines()[1:-1]
        for line in lines:
           data = line.split()
           for x in range(0, len(data), 9):
               data_i.append(int(data[x]))
               data_position.append(float(data[x+1]))
               data_direction.append(float(data[x+2]))
               data_polarization.append(float(data[x+3]))
               data_frequency.append(float(data[x+4]))
               data_freqoff.append(float(data[x+5]))
               data_lastpos.append(float(data[x+6]))
               data_vi.append(float(data[x+7]))
               data_vf.append(float(data[x+8]))
            
    
    
    dataArray = np.zeros((len(data_i),9))
    for i in range(0,len(data_i),1):
        dataArray[i,0]=data_i[i]
        dataArray[i,1]=data_position[i]
        dataArray[i,2]=data_direction[i]
        dataArray[i,3]=data_polarization[i]
        dataArray[i,4]=data_frequency[i]
        dataArray[i,5]=data_freqoff[i]
        dataArray[i,6]=data_lastpos[i]
        dataArray[i,7]=data_vi[i]
        dataArray[i,8]=data_vf[i]
        
       
    c_data = []
    c_j = []
    c_DNHImid = []
    c_y1H = []
    c_y1HE = []
    c_EH = []
    c_Te = []
    c_EHII = []
    c_THII = []
    c_EHI = []
    c_THI = []
    c_EHeI = []
    c_THeI = []
    c_EHeII = []
    c_THeII = []
    c_dEH = []      
    
    emptyline=0
    #here insert the path name to a file with the bottom output matrix from chenxiao's code 
    with open(temperature_path) as f:
        for _ in range(1):
            next(f)
        for line in f:
            if not line.strip():
                #print(emptyline)
                emptyline = emptyline+ 1
              
            if (emptyline ==4):
                next(f)
                next(f)
                next(f)
                next(f)
                emptyline  = emptyline+1
            if (emptyline ==5):
                c_data = line.split()
                for x in range(0, len(c_data), 15):
                    #print(line+ "here")
                    c_j.append(int(c_data[x]))
                    c_DNHImid.append(float(c_data[x+1]))
                    c_y1H.append(float(c_data[x+2]))
                    c_y1HE.append(float(c_data[x+3]))
                    c_EH.append(float(c_data[x+4]))
                    c_Te.append(float(c_data[x+5]))
                    c_EHII.append(float(c_data[x+6]))
                    c_THII.append(float(c_data[x+7]))
                    c_EHI.append(float(c_data[x+8]))
                    c_THI.append(float(c_data[x+9]))
                    c_EHeI.append(float(c_data[x+10]))
                    c_THeI.append(float(c_data[x+11]))
                    c_EHeII.append(float(c_data[x+12]))
                    c_THeII.append(float(c_data[x+13]))
                    c_dEH.append(float(c_data[x+14]))  
               
    cdataArray = np.zeros((len(c_j),15))
    for i in range(0,len(c_j),1):
        cdataArray[i,0]=c_j[i]
        cdataArray[i,1]=c_DNHImid[i]
        cdataArray[i,2]=c_y1H[i]
        cdataArray[i,3]=c_y1HE[i]
        cdataArray[i,4]=c_EH[i]
        cdataArray[i,5]=c_Te[i]
        cdataArray[i,6]=c_EHII[i]
        cdataArray[i,7]=c_THII[i]
        cdataArray[i,8]=c_EHI[i]
        cdataArray[i,9]=c_THI[i]
        cdataArray[i,10]=c_EHeI[i]
        cdataArray[i,11]=c_THeI[i]
        cdataArray[i,12]=c_EHeII[i]
        cdataArray[i,13]=c_THeII[i]
        cdataArray[i,14]=c_dEH[i]
    return dataArray, cdataArray

def give0(x,y):
    if (y==0):
        return 0
    else:
       return x/y 

def LyaObsFun1(vi_index,T_index,nH_index,data,c_data):
    h = 6.6261e-27 #erg s 
    points = 2000000
    vlya = 2.47e15   #in Hz   needs to be the uncorrected one 
    z=8
    vlya_z = vlya/(1+z)
    vobs = np.linspace(vlya_z -1,vlya_z + 1,points)   #make sure centered around redshifted vlya to current age
    theta= 3*np.pi/4  #will be adjusted later ADJUST THIS 
    mu= np.cos(theta) 
    intensity_vobs = []
    Qintensity_vobs = []
    RYD_K = 157800 #rydberg 
    #print("mu ")
    #print(mu)
    n=0  #initialize 
    nH = 1.37e-5 #hydrogen density at redshift    
    fhe = 0.079 # frac of he relative to hydrogen 
    slabwidth = 2.5e16 #FIX THIS
    
    H = 3.3149362e-17 # Hubble constant in the period of z = 8, in [s^-1]
    C= 29979245800 #speed of light, in [cm s^-1]
    
    
    
    
   # rangeR=(-.01,.01)
    if nH_index==1:
        nH=5e-5
       # rangeR=(-.01,.01)
    if nH_index==2:
        nH=1.37e-4
    if nH_index==3:
        nH=5e-4    
    if nH_index==4:
        nH=1.37e-3       
    u = 5e7
    if vi_index==1:
        u = 1e8
    if vi_index ==2:
        u=5e8
    if vi_index ==3:
        u=1e9
    if vi_index==4:
        u=5e9
    
    
    #data = []
    data_i = []
    data_position = []
    data_direction = []
    data_polarization = []
    data_frequency = []
    data_freqoff = []
    data_lastpos = []
    data_vi=[]
    data_vf=[]
    
    #the open function will be modified depend on your own directory
    #with open(scatter_path) as f:
    #    for _ in range(1):
    #        next(f)
    #    for line in f:
    #        data = line.split()
    #print(len(data))
    for x in range(0,len(data), 1):
        data_i.append(int(data[x,0]))
        data_position.append(float(data[x,1]))
        data_direction.append(float(data[x,2]))
        data_polarization.append(float(data[x,3]))
        data_frequency.append(float(data[x,4]))
        data_freqoff.append(float(data[x,5]))
        data_lastpos.append(float(data[x,6]))
        data_vi.append(float(data[x,7]))
        data_vf.append(float(data[x,8]))

         
    
    #c_data = []
    c_j = []
    c_DNHImid = []
    c_y1H = []
    c_y1HE = []
    c_EH = []
    c_Te = []
    c_EHII = []
    c_THII = []
    c_EHI = []
    c_THI = []
    c_EHeI = []
    c_THeI = []
    c_EHeII = []
    c_THeII = []
    c_dEH = []      
    
    #here insert the path name to a file with the bottom output matrix from chenxiao's code 

    for x in range(0, len(c_data), 15):
        c_j.append(int(c_data[x,0]))
        c_DNHImid.append(float(c_data[x,1]))
        c_y1H.append(float(c_data[x,2]))
        c_y1HE.append(float(c_data[x,3]))
        c_EH.append(float(c_data[x,4]))
        c_Te.append(float(c_data[x,5]))
        c_EHII.append(float(c_data[x,6]))
        c_THII.append(float(c_data[x,7]))
        c_EHI.append(float(c_data[x,8]))
        c_THI.append(float(c_data[x,9]))
        c_EHeI.append(float(c_data[x,10]))
        c_THeI.append(float(c_data[x,11]))
        c_EHeII.append(float(c_data[x,12]))
        c_THeII.append(float(c_data[x,13]))
        c_dEH.append(float(c_data[x,14]))
          
    def P0(x):
        return 1
    def P1(x):
        return x
    def P2(x):
        return (3*x**2)/2. - 1/2.
    def P3(x):
        return (5*x**3)/2. - 3*x/2.
    def P4(x):
        return (35*x**4)/8. - (15*x**2)/4. + 3/8.
    def P5(x):
        return (63*x**5)/8. - (35*x**3)/4. + 15*x/8.
    
    def delta(x,epsilon):
        return np.exp(-x*x/(4*epsilon))/(2*np.sqrt(np.pi * epsilon))
    
    def P22(x):
        return 3*(1-x**2)
    
    def P23(x):
        return 15*x*(1-x**2)
    
    def P24(x):
        return 15/2 * (7*x**2-1)*(1-x**2)
    
    def P25(x):
        return 1/4*(11*x*P24(x) - 7*P23(x))
    
    def P26(x):
        return 1/5*(13*x*P25(x) - 8*P24(x))
    
    def P27(x):
        return 1/6*(15*x*P26(x) - 9*P25(x))
    
    def assocLegFit(x,f2,f3,f4,f5,f6,f7):
        return f2*P22(x) + f3*P23(x) + f4*P24(x)+ f5*P25(x) +f6*P26(x)+f7*P27(x)
    
    
    
    mu_0 = []
    mu_1 = []
    mu_2 = []
    mu_3 = []
    mu_4 = []
    mu_5 = []
    
    for x in range(len(data_direction)):
        mu_0.append(P0(data_direction[x]))
        mu_1.append(P1(data_direction[x]))
        mu_2.append(P2(data_direction[x]))
        mu_3.append(P3(data_direction[x]))
        mu_4.append(P4(data_direction[x]))
        mu_5.append(P5(data_direction[x]))
        
    total_0 = sum(mu_0)
    total_1 = sum(mu_1)
    total_2 = sum(mu_2)
    total_3 = sum(mu_3)
    total_4 = sum(mu_4)
    total_5 = sum(mu_5)
    
    b0 = (2*0+1)*total_0/(2*len(data_direction))
    b1 = (2*1+1)*total_1/(2*len(data_direction))
    b2 = (2*2+1)*total_2/(2*len(data_direction))
    b3 = (2*3+1)*total_3/(2*len(data_direction))
    b4 = (2*4+1)*total_4/(2*len(data_direction))
    b5 = (2*5+1)*total_5/(2*len(data_direction))
    
    def P_mu(x):
        prob = (b0 + b1*P1(x) + b2*P2(x) + b3*P3(x) + b4*P4(x) + b5*P5(x))
        if (prob<0):
            return 0 
        else:
            return prob
    x1 = np.arange(-1, 1, 0.00001)
    y1 = np.arange(-1, 1, 0.00001)
    for l in range(0,len(x1),1):
        y1[l] = P_mu(x1[l])
    

    #print("b coeff") 
    #print(b0, b1, b2, b3, b4, b5)
    """
    hist,bin_edges, _ = plt.hist(data_direction, density=True,bins=1000)
    plt.plot(x1, y1)
    plt.legend(['Fitted', 'Simulated'])
    plt.title('Probability density of mu %d %i %u' % (T_index,vi_index,nH_index))
    plt.xlabel('Direction (mu)')
    plt.ylabel('Probability density (P(mu))')
    plt.show()
    """
    
    #photonspike = #send list to yuanyuan after meeting for 234 
    
    
    #freq_std = np.std(data_frequency)
    #in cm^3/s
    def qLya(Te):
        
        # 1s to 2p
        gammaL = 3.35e-1 + 1.45e-5*Te +1.39e-10*Te*Te - 5.66e-15*Te*Te*Te
        if(Te>2.5e4):
            gammaL = 3.28e-1 + 1.43e-5*Te -6.55e-12*Te*Te - 2.69e-18*Te*Te*Te
        q12p = 8.6287e-6/2./np.sqrt(Te)*gammaL*np.exp(-0.75*RYD_K/Te)
        
        # 1s to 3s
        gammaL = 8.27e-2 + 4.18e-6*Te -4.18e-10*Te*Te +9.37e-15*Te*Te*Te
        if (Te>2.5e4):
            gammaL= 7.31e-2 - 1.84e-7*Te +1.31e-12*Te*Te -1.62e-18*Te*Te*Te
        q13s = 8.6287e-6/2./np.sqrt(Te)*gammaL*np.exp(-0.8888888889*RYD_K/Te)
        
        # 1s to 3d
        gammaL = 1.21e-2 + 1.03e-5*Te -4.31e-10*Te*Te + 7.16e-15*Te*Te*Te
        if (Te>2.5e4):
            gammaL = 9.34e-2 + 1.03e-6*Te -3.68e-12*Te*Te +3.91e-18*Te*Te*Te
        q13d = 8.6287e-6/2./np.sqrt(Te)*gammaL*np.exp(-0.8888888889*RYD_K/Te)
        
        return (q12p + q13s + q13d)
        
    
    
    for j in range(0,len(c_j),1):
        n = n + slabwidth*nH*qLya(c_Te[j])*c_y1H[j]*(1-c_y1H[j] + fhe*(1-c_y1HE[j])) 
        #if (j%100 ==0):
         #   print(qLya(c_Te[j]))     
    
    p_mu = P_mu(mu)
    count1= 0
    ind=0
    """
    while (count1 == 0):
        if (bin_edges[ind]<mu and bin_edges[ind+1]>mu):
            count1 = count1+1
        ind = ind + 1  
        #print(ind)
    #print(bin_edges[ind-1])
    #print(bin_edges[ind])
    p_mu = hist[ind-1]
    #print('intensity')
    """
    intensity = h*vlya_z*n*p_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3)
   # print(intensity)
    #for i in range(0,points,1):
    #    intensity_vobs_add = delta(vobs[i]-vlya_z,.001)*h*vobs[i]*n*p_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3) 
    #    intensity_vobs.append(intensity_vobs_add)
    
    #delta(vobs[i]-vlya_z,.001)*
    """
    plt.plot(np.array(vobs),np.array(intensity_vobs))
    plt.xlim(-1+vlya_z, 1+vlya_z)
    plt.xlabel('frequency')
    plt.ylabel('Intensity(v)')
    plt.title('intensity curve as seen by observer %d %i %u' % (vi_index,T_index,nH_index) )
    plt.show()
    """
    
    #code to also give Q 
    
    
    # for Q, can we multiply the intesnsity by the polarization to give the directional affect 
    
    polar0 = []
    polar1 = []
    polar2 = []
    polar3 = []
    polar4 = []
    polar5 = []
    polar6 = []
    polar7 = []
    polar8 = []
    polar9 = []
    
    #vobsZ = np.linspace(np.min(data_frequency,max(data_frequency),points))
    
                        
    
    for x in range(len(data_direction)):
        if data_direction[x] < -0.8:
            polar0.append(data_polarization[x])
        if data_direction[x] >= -0.8 and data_direction[x] < -0.6:
            polar1.append(data_polarization[x])
        if data_direction[x] >= -0.6 and data_direction[x] < -0.4:
            polar2.append(data_polarization[x])
        if data_direction[x] >= -0.4 and data_direction[x] < -0.2:
            polar3.append(data_polarization[x])
        if data_direction[x] >= -0.2 and data_direction[x] < 0.0:
            polar4.append(data_polarization[x])
        if data_direction[x] >= 0.0 and data_direction[x] < 0.2:
            polar5.append(data_polarization[x])
        if data_direction[x] >= 0.2 and data_direction[x] < 0.4:
            polar6.append(data_polarization[x])
        if data_direction[x] >= 0.4 and data_direction[x] < 0.6:
            polar7.append(data_polarization[x])
        if data_direction[x] >= 0.6 and data_direction[x] < 0.8:
            polar8.append(data_polarization[x])
        if data_direction[x] >= 0.8 and data_direction[x] < 1.0:
            polar9.append(data_polarization[x])
   
    p0 = sum(polar0)/len(data_polarization)/0.2
    p1 = sum(polar1)/len(data_polarization)/0.2
    p2 = sum(polar2)/len(data_polarization)/0.2
    p3 = sum(polar3)/len(data_polarization)/0.2
    p4 = sum(polar4)/len(data_polarization)/0.2
    p5 = sum(polar5)/len(data_polarization)/0.2
    p6 = sum(polar6)/len(data_polarization)/0.2
    p7 = sum(polar7)/len(data_polarization)/0.2
    p8 = sum(polar8)/len(data_polarization)/0.2
    p9 = sum(polar9)/len(data_polarization)/0.2
    
    x2 = [-0.9, -0.7, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 0.7, 0.9]
    #y2 = [p0_avg, p1_avg, p2_avg, p3_avg, p4_avg, p5_avg, p6_avg, p7_avg, p8_avg, p9_avg]
    y3 = [p0, p1, p2, p3, p4, p5, p6, p7, p8, p9]
    """
    plt.plot(x2, y3)
    plt.title('Mean polarization in each direction bin')
    plt.xlabel('Direction (mu)')
    plt.ylabel('Probability density')
    plt.show()
    """
        
    #want to fit <q/i>P(mu) using associated lp, have polarization(mu) and Prob(mu)
    qIP2 = []
    qIP3 = []
    qIP4 = []
    qIP5 = []
    qIP6 = []   
    qIP7 = []
    histogram = np.zeros(len(data_direction))
    
    for x in range(len(data_direction)):
        histogram[x] = (data_direction[x]*data_polarization[x])
        qIP2.append(P22(data_direction[x]*data_polarization[x]))
        qIP3.append(P23(data_direction[x]*data_polarization[x]))
        qIP4.append(P24(data_direction[x]*data_polarization[x]))
        qIP5.append(P25(data_direction[x]*data_polarization[x]))
        qIP6.append(P26(data_direction[x]*data_polarization[x]))
        qIP7.append(P27(data_direction[x]*data_polarization[x]))
        
    
    totalq2 =sum(qIP2)
    totalq3 =sum(qIP3)
    totalq4 =sum(qIP4)
    totalq5 =sum(qIP5)
    totalq6 =sum(qIP6)
    totalq7 =sum(qIP7)
    
    totalQ = totalq2+totalq3+totalq4+totalq5+totalq6+totalq7 
    
    
    
    #popt, _ = curve_fit(assocLegFit, data_direction, histogram)
    #cf2,cf3,cf4,cf5,cf6,cf7 = popt
  
    #try again with weight of p in bin 
    
    q10 = []
    q11 = []
    q12 = []
    q13 = []
    q14 = []
    q15 = []
    q16 = []
    q17 = []
    q18 = []
    q19 = []
    
    
    for x in range(len(data_direction)):
        if data_direction[x] < -0.8:
            q10.append(P_mu(data_direction[x])*p0)
        if data_direction[x] >= -0.8 and data_direction[x] < -0.6:
            q11.append(P_mu(data_direction[x])*p1)
        if data_direction[x] >= -0.6 and data_direction[x] < -0.4:
            q12.append(P_mu(data_direction[x])*p2)
        if data_direction[x] >= -0.4 and data_direction[x] < -0.2:
            q13.append(P_mu(data_direction[x])*p3)
        if data_direction[x] >= -0.2 and data_direction[x] < 0.0:
            q14.append(P_mu(data_direction[x])*p4)
        if data_direction[x] >= 0.0 and data_direction[x] < 0.2:
            q15.append(P_mu(data_direction[x])*p5)
        if data_direction[x] >= 0.2 and data_direction[x] < 0.4:
            q16.append(P_mu(data_direction[x])*p6)
        if data_direction[x] >= 0.4 and data_direction[x] < 0.6:
            q17.append(P_mu(data_direction[x])*p7)
        if data_direction[x] >= 0.6 and data_direction[x] < 0.8:
            q18.append(P_mu(data_direction[x])*p8)
        if data_direction[x] >= 0.8 and data_direction[x] < 1.0:
            q19.append(P_mu(data_direction[x])*p9)
    
    
    w0 = give0(sum(q10),len(q10))
    w1 = give0(sum(q11),len(q11))
    w2 = give0(sum(q12),len(q12))
    w3 = give0(sum(q13),len(q13))
    w4 = give0(sum(q14),len(q14))
    w5 = give0(sum(q15),len(q15))
    w6 = give0(sum(q16),len(q16))
    w7 = give0(sum(q17),len(q17))
    w8 =give0(sum(q18),len(q18))
    w9 = give0(sum(q19),len(q19))
    
    yw = [w0,w1,w2,w3,w4,w5,w6,w7,w8,w9]
    ywarray = np.array(yw)
    x2array = np.array(x2)
    x1array = np.array(x1)
    
    popt, _ = curve_fit(assocLegFit, x2array, ywarray)
    cf2a,cf3a,cf4a,cf5a,cf6a, cf7a = popt
    
    yQ1 = cf2a*P22(x1array) + cf3a*P23(x1array) + cf4a*P24(x1array)+ cf5a*P25(x1array) + cf6a*P26(x1array) + cf7a*P27(x1array)
    yQ2 = cf2a*P22(x2array) + cf3a*P23(x2array) + cf4a*P24(x2array)+ cf5a*P25(x2array) + cf6a*P26(x2array) + cf7a*P27(x2array)
    """
    plt.hist(histogram, density=True)
    plt.title('Probability density of mu*polarization')
    plt.xlabel('Direction (mu)')
    plt.ylabel('<Q/I>P(mu)')
    plt.show()
    
    #plt.plot(x2,yw)
    plt.plot(x2,yQ2)
    plt.plot(x1,yQ1)
    plt.title('P(mu)*polarization')
    plt.xlabel('Direction (mu)')
    plt.ylabel('P(mu)*polarization')
    plt.legend(['Simulated','Fitted'])
    plt.show()
    """
    #YQ1 works!!!!!! move on to writing Q 
    
    def PQI_mu(x):
        return assocLegFit(x, cf2a, cf3a, cf4a, cf5a, cf6a, cf7a)
    
    
    pQI_mu = PQI_mu(mu)  
   # print("polarized intensity") 
    polarizedIntensity = h*vlya_z*n*pQI_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3)
   # print(polarizedIntensity)
    #for i in range(0,points,1):
    #    Qintensity_vobs_add = delta(vobs[i]-vlya_z,.001)*h*vobs[i]*n*pQI_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3) 
    #    Qintensity_vobs.append(Qintensity_vobs_add)
    
    """
    plt.plot(vobs,Qintensity_vobs)
    plt.xlim(-1+vlya_z, 1+vlya_z)
    plt.xlabel("frequency")
    plt.ylabel("Polarized Intensity(v)")
    plt.title("Polarized intensity %d %i %u" % (vi_index,T_index,nH_index))
    plt.show()
    """
    
    """
    plt.plot(data_direction,data_lastpos,".")
    plt.title("Position of Last Scatterign")
    plt.show()
    
    plt.plot(data_direction,data_frequency,".")
    plt.title("frequency")
    plt.show()
    
    plt.plot(data_direction,data_position,".")
    plt.title("Last Position")
    plt.show()
    """
    """
    ### For Extrap Hist
    
    frontLoc = findFrontLocation(cdataArray[:,2]) 
    print(frontLoc)
    
    fracMuExtrap = []
    tail=[]
    print("extreme extrap")
    countd=0
    for ind in range(0,len(data_i),1):
        #print("new")
        a= (data_vf[ind]-data_frequency[ind])/vlya 
        #print(a)
        b= H*(slabwidth*frontLoc - slabwidth*data_lastpos[ind])/(nH*(C*data_direction[ind] - u))
        #print(b)
        fracMu =a + b
        #print(fracMu)
        #fracMuExtrap.append(fracMu)
        if (np.abs(fracMu)>1):
            print(ind)
            print(fracMu)
            print("Denominator " + str(C*data_direction[ind] - u))
            tail.append(ind)
        else:
            if (countd<5):
                print(fracMu)
                print("Denominator " + str(C*data_direction[ind] - u))
                countd+=1
            fracMuExtrap.append(fracMu)
            
    sigmafracmu = np.std(fracMuExtrap)       
    print("STD is " + str(sigmafracmu))
    """
    

    """
    n, bins, patches = plt.hist(fracMuExtrap,1500)
    plt.title(str(T_index)+str(vi_index) +str(nH_index,))
    plt.xlim(-2.5*sigmafracmu,2.5*sigmafracmu)
    plt.show()        
    """            
    return (intensity, polarizedIntensity) #fracMuExtrap,tail)

def runsDataForErrors(ii,j,k,dataArray,cdataArray):
    
    print("started")
    
    with NumpyRNGContext(1):
        bootresult = bootstrap(dataArray, 1000)

    print("here")    

    bootstrapIntensities =[]
    bootstrapPolIntensities = []
    for w in range(0,len(bootresult),1):
        x,y = LyaObsFun1(ii,j,k,bootresult[w],cdataArray)
        bootstrapIntensities.append(x)
        bootstrapPolIntensities.append(y)
        
    """    
    plt.hist(bootstrapIntensities,30)
    plt.title("Intensities" + str(ii)+str(j)+str(k) + " n=1000")  
    plt.show()  
    plt.hist(bootstrapPolIntensities,30)
    plt.title("PolIntensities "+ str(ii)+str(j)+str(k) + " n=1000") 
    plt.show()
    """
    
    
    mean = np.mean(bootstrapIntensities)
    
    std=np.std(bootstrapIntensities)
    
    meanQ = np.mean(bootstrapPolIntensities)
    
    stdQ=np.std(bootstrapPolIntensities)
    #Intensities222 = np.array(bootstrapIntensities222)
    #PolIntensities222 = np.array(bootstrapPolIntensities222)
    return mean, meanQ, std, stdQ

directory = "/fs/project/PCON0003/yuanyuan/reionization/lastest_test/"
gridsize = 5
Intenlist = []
devIlist = []
PolIntensList =[]
devQList =[]
with open('EmilyOutputs2923_2.txt', 'w') as f:
    f.write("Intensity \t Error \t PolarizedIntensity \t ErrorPol \n " )
    for i in range(0,5,1):
        for j in range(2,5,1):
            for k in range(0,5,1):
                run = str(i)+str(j)+str(k)
                if (run != "445"):
                    print(run)
                    scatterpath1 = directory +"lya_" + run+".txt"
                    #scatterpath2 = directory +"lya_" + run+"_2.txt"
                    temppath = directory +"output_" + str(i)+str(j)+".txt"
                    #if(run != '200' and run !="210"):
                    (dataArray,cdataArray)=  getFileDataArrays(i,j,k,scatterpath1,temppath) 
                    #intensity,polintensity,fracMu,tail = LyaObsFun1(j, i, k, dataArray, cdataArray)
                    mean,meanQ,std,stdQ = runsDataForErrors(i,j,k,dataArray,cdataArray)
                    print("intensity= " + str(mean))
                    print("std= " + str(std))
                    print("pol intensity= " + str(meanQ))
                    print()
                    print("stdPol= "+ str(stdQ))
                    f.write(str(mean)+ "\t" + str(std) + "\t" + str(meanQ) + "\t" + str(stdQ) + "\n")
                    Intenlist.append(mean)
                    PolIntensList.append(meanQ)
                    devIlist.append(std)
                    devQList.append(stdQ)
                   # nlist.append(x)
                   # if (run == "112"):
                       # taillist.append(y)    

f.close()


020
started
here
intensity= 9.481658166169617e-15
std= 5.3009805491496596e-17
pol intensity= 5.703895802855473e-18

stdPol= 2.0093129354296963e-17
021
started
here
intensity= 5.175154455159611e-14
std= 1.9906148226961697e-16
pol intensity= 1.3737052749172414e-15

stdPol= 1.3780816193723447e-16
022
started
here
intensity= 1.7237656538946017e-13
std= 5.801771061970647e-16
pol intensity= 6.172443576763912e-15

stdPol= 4.918587259432467e-16
023
started
here
intensity= 6.778279305861094e-13
std= 2.398182317765551e-15
pol intensity= 2.11021417139503e-14

stdPol= 2.1475493594043758e-15
024
started
here
intensity= 1.862861374508985e-12
std= 7.87373289964048e-15
pol intensity= 4.7807893929930516e-14

stdPol= 7.13344362686299e-15
030
started
here
intensity= 2.1012385564163977e-14
std= 1.163790350358366e-16
pol intensity= 2.0252252010822792e-16

stdPol= 4.7983255227745196e-17
031
started
here
intensity= 1.1663165837388258e-13
std= 4.616846160075533e-16
pol intensity= 3.1470252778266155e-15

stdPo

In [ ]:

import numpy as np
from astropy.stats import bootstrap
from astropy.utils import NumpyRNGContext
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def findFrontLocation(fracH):
    elementLocation = 0
    i=0
    while elementLocation == 0:
        if (fracH[i]-.5) > 0 :
            elementLocation=i
        i = i+1     
    return elementLocation  


def getFileDataArrays(i,j,k,scatter_path1,temperature_path):
    data = []
    data_i = []
    data_position = []
    data_direction = []
    data_polarization = []
    data_frequency = []
    data_freqoff = []
    data_lastpos = []
    data_vi=[]
    data_vf=[]
    
    
    with open(scatter_path1) as f:
        lines = f.readlines()[1:-1]
        for line in lines:
           data = line.split()
           for x in range(0, len(data), 9):
               data_i.append(int(data[x]))
               data_position.append(float(data[x+1]))
               data_direction.append(float(data[x+2]))
               data_polarization.append(float(data[x+3]))
               data_frequency.append(float(data[x+4]))
               data_freqoff.append(float(data[x+5]))
               data_lastpos.append(float(data[x+6]))
               data_vi.append(float(data[x+7]))
               data_vf.append(float(data[x+8]))
            
    
    
    dataArray = np.zeros((len(data_i),9))
    for i in range(0,len(data_i),1):
        dataArray[i,0]=data_i[i]
        dataArray[i,1]=data_position[i]
        dataArray[i,2]=data_direction[i]
        dataArray[i,3]=data_polarization[i]
        dataArray[i,4]=data_frequency[i]
        dataArray[i,5]=data_freqoff[i]
        dataArray[i,6]=data_lastpos[i]
        dataArray[i,7]=data_vi[i]
        dataArray[i,8]=data_vf[i]
        
       
    c_data = []
    c_j = []
    c_DNHImid = []
    c_y1H = []
    c_y1HE = []
    c_EH = []
    c_Te = []
    c_EHII = []
    c_THII = []
    c_EHI = []
    c_THI = []
    c_EHeI = []
    c_THeI = []
    c_EHeII = []
    c_THeII = []
    c_dEH = []      
    
    emptyline=0
    #here insert the path name to a file with the bottom output matrix from chenxiao's code 
    with open(temperature_path) as f:
        for _ in range(1):
            next(f)
        for line in f:
            if not line.strip():
                #print(emptyline)
                emptyline = emptyline+ 1
              
            if (emptyline ==4):
                next(f)
                next(f)
                next(f)
                next(f)
                emptyline  = emptyline+1
            if (emptyline ==5):
                c_data = line.split()
                for x in range(0, len(c_data), 15):
                    #print(line+ "here")
                    c_j.append(int(c_data[x]))
                    c_DNHImid.append(float(c_data[x+1]))
                    c_y1H.append(float(c_data[x+2]))
                    c_y1HE.append(float(c_data[x+3]))
                    c_EH.append(float(c_data[x+4]))
                    c_Te.append(float(c_data[x+5]))
                    c_EHII.append(float(c_data[x+6]))
                    c_THII.append(float(c_data[x+7]))
                    c_EHI.append(float(c_data[x+8]))
                    c_THI.append(float(c_data[x+9]))
                    c_EHeI.append(float(c_data[x+10]))
                    c_THeI.append(float(c_data[x+11]))
                    c_EHeII.append(float(c_data[x+12]))
                    c_THeII.append(float(c_data[x+13]))
                    c_dEH.append(float(c_data[x+14]))  
               
    cdataArray = np.zeros((len(c_j),15))
    for i in range(0,len(c_j),1):
        cdataArray[i,0]=c_j[i]
        cdataArray[i,1]=c_DNHImid[i]
        cdataArray[i,2]=c_y1H[i]
        cdataArray[i,3]=c_y1HE[i]
        cdataArray[i,4]=c_EH[i]
        cdataArray[i,5]=c_Te[i]
        cdataArray[i,6]=c_EHII[i]
        cdataArray[i,7]=c_THII[i]
        cdataArray[i,8]=c_EHI[i]
        cdataArray[i,9]=c_THI[i]
        cdataArray[i,10]=c_EHeI[i]
        cdataArray[i,11]=c_THeI[i]
        cdataArray[i,12]=c_EHeII[i]
        cdataArray[i,13]=c_THeII[i]
        cdataArray[i,14]=c_dEH[i]
    return dataArray, cdataArray

def give0(x,y):
    if (y==0):
        return 0
    else:
       return x/y 

def LyaObsFun1(vi_index,T_index,nH_index,data,c_data):
    h = 6.6261e-27 #erg s 
    points = 2000000
    vlya = 2.47e15   #in Hz   needs to be the uncorrected one 
    z=8
    vlya_z = vlya/(1+z)
    vobs = np.linspace(vlya_z -1,vlya_z + 1,points)   #make sure centered around redshifted vlya to current age
    theta= 3*np.pi/4  #will be adjusted later ADJUST THIS 
    mu= np.cos(theta) 
    intensity_vobs = []
    Qintensity_vobs = []
    RYD_K = 157800 #rydberg 
    #print("mu ")
    #print(mu)
    n=0  #initialize 
    nH = 1.37e-5 #hydrogen density at redshift    
    fhe = 0.079 # frac of he relative to hydrogen 
    slabwidth = 2.5e16 #FIX THIS
    
    H = 3.3149362e-17 # Hubble constant in the period of z = 8, in [s^-1]
    C= 29979245800 #speed of light, in [cm s^-1]
    
    
    
    
   # rangeR=(-.01,.01)
    if nH_index==1:
        nH=5e-5
       # rangeR=(-.01,.01)
    if nH_index==2:
        nH=1.37e-4
    if nH_index==3:
        nH=5e-4    
    if nH_index==4:
        nH=1.37e-3       
    u = 5e7
    if vi_index==1:
        u = 1e8
    if vi_index ==2:
        u=5e8
    if vi_index ==3:
        u=1e9
    if vi_index==4:
        u=5e9
    
    
    #data = []
    data_i = []
    data_position = []
    data_direction = []
    data_polarization = []
    data_frequency = []
    data_freqoff = []
    data_lastpos = []
    data_vi=[]
    data_vf=[]
    
    #the open function will be modified depend on your own directory
    #with open(scatter_path) as f:
    #    for _ in range(1):
    #        next(f)
    #    for line in f:
    #        data = line.split()
    #print(len(data))
    for x in range(0,len(data), 1):
        data_i.append(int(data[x,0]))
        data_position.append(float(data[x,1]))
        data_direction.append(float(data[x,2]))
        data_polarization.append(float(data[x,3]))
        data_frequency.append(float(data[x,4]))
        data_freqoff.append(float(data[x,5]))
        data_lastpos.append(float(data[x,6]))
        data_vi.append(float(data[x,7]))
        data_vf.append(float(data[x,8]))

         
    
    #c_data = []
    c_j = []
    c_DNHImid = []
    c_y1H = []
    c_y1HE = []
    c_EH = []
    c_Te = []
    c_EHII = []
    c_THII = []
    c_EHI = []
    c_THI = []
    c_EHeI = []
    c_THeI = []
    c_EHeII = []
    c_THeII = []
    c_dEH = []      
    
    #here insert the path name to a file with the bottom output matrix from chenxiao's code 

    for x in range(0, len(c_data), 15):
        c_j.append(int(c_data[x,0]))
        c_DNHImid.append(float(c_data[x,1]))
        c_y1H.append(float(c_data[x,2]))
        c_y1HE.append(float(c_data[x,3]))
        c_EH.append(float(c_data[x,4]))
        c_Te.append(float(c_data[x,5]))
        c_EHII.append(float(c_data[x,6]))
        c_THII.append(float(c_data[x,7]))
        c_EHI.append(float(c_data[x,8]))
        c_THI.append(float(c_data[x,9]))
        c_EHeI.append(float(c_data[x,10]))
        c_THeI.append(float(c_data[x,11]))
        c_EHeII.append(float(c_data[x,12]))
        c_THeII.append(float(c_data[x,13]))
        c_dEH.append(float(c_data[x,14]))
          
    def P0(x):
        return 1
    def P1(x):
        return x
    def P2(x):
        return (3*x**2)/2. - 1/2.
    def P3(x):
        return (5*x**3)/2. - 3*x/2.
    def P4(x):
        return (35*x**4)/8. - (15*x**2)/4. + 3/8.
    def P5(x):
        return (63*x**5)/8. - (35*x**3)/4. + 15*x/8.
    
    def delta(x,epsilon):
        return np.exp(-x*x/(4*epsilon))/(2*np.sqrt(np.pi * epsilon))
    
    def P22(x):
        return 3*(1-x**2)
    
    def P23(x):
        return 15*x*(1-x**2)
    
    def P24(x):
        return 15/2 * (7*x**2-1)*(1-x**2)
    
    def P25(x):
        return 1/4*(11*x*P24(x) - 7*P23(x))
    
    def P26(x):
        return 1/5*(13*x*P25(x) - 8*P24(x))
    
    def P27(x):
        return 1/6*(15*x*P26(x) - 9*P25(x))
    
    def assocLegFit(x,f2,f3,f4,f5,f6,f7):
        return f2*P22(x) + f3*P23(x) + f4*P24(x)+ f5*P25(x) +f6*P26(x)+f7*P27(x)
    
    
    
    mu_0 = []
    mu_1 = []
    mu_2 = []
    mu_3 = []
    mu_4 = []
    mu_5 = []
    
    for x in range(len(data_direction)):
        mu_0.append(P0(data_direction[x]))
        mu_1.append(P1(data_direction[x]))
        mu_2.append(P2(data_direction[x]))
        mu_3.append(P3(data_direction[x]))
        mu_4.append(P4(data_direction[x]))
        mu_5.append(P5(data_direction[x]))
        
    total_0 = sum(mu_0)
    total_1 = sum(mu_1)
    total_2 = sum(mu_2)
    total_3 = sum(mu_3)
    total_4 = sum(mu_4)
    total_5 = sum(mu_5)
    
    b0 = (2*0+1)*total_0/(2*len(data_direction))
    b1 = (2*1+1)*total_1/(2*len(data_direction))
    b2 = (2*2+1)*total_2/(2*len(data_direction))
    b3 = (2*3+1)*total_3/(2*len(data_direction))
    b4 = (2*4+1)*total_4/(2*len(data_direction))
    b5 = (2*5+1)*total_5/(2*len(data_direction))
    
    def P_mu(x):
        prob = (b0 + b1*P1(x) + b2*P2(x) + b3*P3(x) + b4*P4(x) + b5*P5(x))
        if (prob<0):
            return 0 
        else:
            return prob
    x1 = np.arange(-1, 1, 0.00001)
    y1 = np.arange(-1, 1, 0.00001)
    for l in range(0,len(x1),1):
        y1[l] = P_mu(x1[l])
    

    #print("b coeff") 
    #print(b0, b1, b2, b3, b4, b5)
    """
    hist,bin_edges, _ = plt.hist(data_direction, density=True,bins=1000)
    plt.plot(x1, y1)
    plt.legend(['Fitted', 'Simulated'])
    plt.title('Probability density of mu %d %i %u' % (T_index,vi_index,nH_index))
    plt.xlabel('Direction (mu)')
    plt.ylabel('Probability density (P(mu))')
    plt.show()
    """
    
    #photonspike = #send list to yuanyuan after meeting for 234 
    
    
    #freq_std = np.std(data_frequency)
    #in cm^3/s
    def qLya(Te):
        
        # 1s to 2p
        gammaL = 3.35e-1 + 1.45e-5*Te +1.39e-10*Te*Te - 5.66e-15*Te*Te*Te
        if(Te>2.5e4):
            gammaL = 3.28e-1 + 1.43e-5*Te -6.55e-12*Te*Te - 2.69e-18*Te*Te*Te
        q12p = 8.6287e-6/2./np.sqrt(Te)*gammaL*np.exp(-0.75*RYD_K/Te)
        
        # 1s to 3s
        gammaL = 8.27e-2 + 4.18e-6*Te -4.18e-10*Te*Te +9.37e-15*Te*Te*Te
        if (Te>2.5e4):
            gammaL= 7.31e-2 - 1.84e-7*Te +1.31e-12*Te*Te -1.62e-18*Te*Te*Te
        q13s = 8.6287e-6/2./np.sqrt(Te)*gammaL*np.exp(-0.8888888889*RYD_K/Te)
        
        # 1s to 3d
        gammaL = 1.21e-2 + 1.03e-5*Te -4.31e-10*Te*Te + 7.16e-15*Te*Te*Te
        if (Te>2.5e4):
            gammaL = 9.34e-2 + 1.03e-6*Te -3.68e-12*Te*Te +3.91e-18*Te*Te*Te
        q13d = 8.6287e-6/2./np.sqrt(Te)*gammaL*np.exp(-0.8888888889*RYD_K/Te)
        
        return (q12p + q13s + q13d)
        
    
    
    for j in range(0,len(c_j),1):
        n = n + slabwidth*nH*qLya(c_Te[j])*c_y1H[j]*(1-c_y1H[j] + fhe*(1-c_y1HE[j])) 
        #if (j%100 ==0):
         #   print(qLya(c_Te[j]))     
    
    p_mu = P_mu(mu)
    count1= 0
    ind=0
    """
    while (count1 == 0):
        if (bin_edges[ind]<mu and bin_edges[ind+1]>mu):
            count1 = count1+1
        ind = ind + 1  
        #print(ind)
    #print(bin_edges[ind-1])
    #print(bin_edges[ind])
    p_mu = hist[ind-1]
    #print('intensity')
    """
    intensity = h*vlya_z*n*p_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3)
   # print(intensity)
    #for i in range(0,points,1):
    #    intensity_vobs_add = delta(vobs[i]-vlya_z,.001)*h*vobs[i]*n*p_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3) 
    #    intensity_vobs.append(intensity_vobs_add)
    
    #delta(vobs[i]-vlya_z,.001)*
    """
    plt.plot(np.array(vobs),np.array(intensity_vobs))
    plt.xlim(-1+vlya_z, 1+vlya_z)
    plt.xlabel('frequency')
    plt.ylabel('Intensity(v)')
    plt.title('intensity curve as seen by observer %d %i %u' % (vi_index,T_index,nH_index) )
    plt.show()
    """
    
    #code to also give Q 
    
    
    # for Q, can we multiply the intesnsity by the polarization to give the directional affect 
    
    polar0 = []
    polar1 = []
    polar2 = []
    polar3 = []
    polar4 = []
    polar5 = []
    polar6 = []
    polar7 = []
    polar8 = []
    polar9 = []
    
    #vobsZ = np.linspace(np.min(data_frequency,max(data_frequency),points))
    
                        
    
    for x in range(len(data_direction)):
        if data_direction[x] < -0.8:
            polar0.append(data_polarization[x])
        if data_direction[x] >= -0.8 and data_direction[x] < -0.6:
            polar1.append(data_polarization[x])
        if data_direction[x] >= -0.6 and data_direction[x] < -0.4:
            polar2.append(data_polarization[x])
        if data_direction[x] >= -0.4 and data_direction[x] < -0.2:
            polar3.append(data_polarization[x])
        if data_direction[x] >= -0.2 and data_direction[x] < 0.0:
            polar4.append(data_polarization[x])
        if data_direction[x] >= 0.0 and data_direction[x] < 0.2:
            polar5.append(data_polarization[x])
        if data_direction[x] >= 0.2 and data_direction[x] < 0.4:
            polar6.append(data_polarization[x])
        if data_direction[x] >= 0.4 and data_direction[x] < 0.6:
            polar7.append(data_polarization[x])
        if data_direction[x] >= 0.6 and data_direction[x] < 0.8:
            polar8.append(data_polarization[x])
        if data_direction[x] >= 0.8 and data_direction[x] < 1.0:
            polar9.append(data_polarization[x])
   
    p0 = sum(polar0)/len(data_polarization)/0.2
    p1 = sum(polar1)/len(data_polarization)/0.2
    p2 = sum(polar2)/len(data_polarization)/0.2
    p3 = sum(polar3)/len(data_polarization)/0.2
    p4 = sum(polar4)/len(data_polarization)/0.2
    p5 = sum(polar5)/len(data_polarization)/0.2
    p6 = sum(polar6)/len(data_polarization)/0.2
    p7 = sum(polar7)/len(data_polarization)/0.2
    p8 = sum(polar8)/len(data_polarization)/0.2
    p9 = sum(polar9)/len(data_polarization)/0.2
    
    x2 = [-0.9, -0.7, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 0.7, 0.9]
    #y2 = [p0_avg, p1_avg, p2_avg, p3_avg, p4_avg, p5_avg, p6_avg, p7_avg, p8_avg, p9_avg]
    y3 = [p0, p1, p2, p3, p4, p5, p6, p7, p8, p9]
    """
    plt.plot(x2, y3)
    plt.title('Mean polarization in each direction bin')
    plt.xlabel('Direction (mu)')
    plt.ylabel('Probability density')
    plt.show()
    """
        
    #want to fit <q/i>P(mu) using associated lp, have polarization(mu) and Prob(mu)
    qIP2 = []
    qIP3 = []
    qIP4 = []
    qIP5 = []
    qIP6 = []   
    qIP7 = []
    histogram = np.zeros(len(data_direction))
    
    for x in range(len(data_direction)):
        histogram[x] = (data_direction[x]*data_polarization[x])
        qIP2.append(P22(data_direction[x]*data_polarization[x]))
        qIP3.append(P23(data_direction[x]*data_polarization[x]))
        qIP4.append(P24(data_direction[x]*data_polarization[x]))
        qIP5.append(P25(data_direction[x]*data_polarization[x]))
        qIP6.append(P26(data_direction[x]*data_polarization[x]))
        qIP7.append(P27(data_direction[x]*data_polarization[x]))
        
    
    totalq2 =sum(qIP2)
    totalq3 =sum(qIP3)
    totalq4 =sum(qIP4)
    totalq5 =sum(qIP5)
    totalq6 =sum(qIP6)
    totalq7 =sum(qIP7)
    
    totalQ = totalq2+totalq3+totalq4+totalq5+totalq6+totalq7 
    
    
    
    #popt, _ = curve_fit(assocLegFit, data_direction, histogram)
    #cf2,cf3,cf4,cf5,cf6,cf7 = popt
  
    #try again with weight of p in bin 
    
    q10 = []
    q11 = []
    q12 = []
    q13 = []
    q14 = []
    q15 = []
    q16 = []
    q17 = []
    q18 = []
    q19 = []
    
    
    for x in range(len(data_direction)):
        if data_direction[x] < -0.8:
            q10.append(P_mu(data_direction[x])*p0)
        if data_direction[x] >= -0.8 and data_direction[x] < -0.6:
            q11.append(P_mu(data_direction[x])*p1)
        if data_direction[x] >= -0.6 and data_direction[x] < -0.4:
            q12.append(P_mu(data_direction[x])*p2)
        if data_direction[x] >= -0.4 and data_direction[x] < -0.2:
            q13.append(P_mu(data_direction[x])*p3)
        if data_direction[x] >= -0.2 and data_direction[x] < 0.0:
            q14.append(P_mu(data_direction[x])*p4)
        if data_direction[x] >= 0.0 and data_direction[x] < 0.2:
            q15.append(P_mu(data_direction[x])*p5)
        if data_direction[x] >= 0.2 and data_direction[x] < 0.4:
            q16.append(P_mu(data_direction[x])*p6)
        if data_direction[x] >= 0.4 and data_direction[x] < 0.6:
            q17.append(P_mu(data_direction[x])*p7)
        if data_direction[x] >= 0.6 and data_direction[x] < 0.8:
            q18.append(P_mu(data_direction[x])*p8)
        if data_direction[x] >= 0.8 and data_direction[x] < 1.0:
            q19.append(P_mu(data_direction[x])*p9)
    
    
    w0 = give0(sum(q10),len(q10))
    w1 = give0(sum(q11),len(q11))
    w2 = give0(sum(q12),len(q12))
    w3 = give0(sum(q13),len(q13))
    w4 = give0(sum(q14),len(q14))
    w5 = give0(sum(q15),len(q15))
    w6 = give0(sum(q16),len(q16))
    w7 = give0(sum(q17),len(q17))
    w8 =give0(sum(q18),len(q18))
    w9 = give0(sum(q19),len(q19))
    
    yw = [w0,w1,w2,w3,w4,w5,w6,w7,w8,w9]
    ywarray = np.array(yw)
    x2array = np.array(x2)
    x1array = np.array(x1)
    
    popt, _ = curve_fit(assocLegFit, x2array, ywarray)
    cf2a,cf3a,cf4a,cf5a,cf6a, cf7a = popt
    
    yQ1 = cf2a*P22(x1array) + cf3a*P23(x1array) + cf4a*P24(x1array)+ cf5a*P25(x1array) + cf6a*P26(x1array) + cf7a*P27(x1array)
    yQ2 = cf2a*P22(x2array) + cf3a*P23(x2array) + cf4a*P24(x2array)+ cf5a*P25(x2array) + cf6a*P26(x2array) + cf7a*P27(x2array)
    """
    plt.hist(histogram, density=True)
    plt.title('Probability density of mu*polarization')
    plt.xlabel('Direction (mu)')
    plt.ylabel('<Q/I>P(mu)')
    plt.show()
    
    #plt.plot(x2,yw)
    plt.plot(x2,yQ2)
    plt.plot(x1,yQ1)
    plt.title('P(mu)*polarization')
    plt.xlabel('Direction (mu)')
    plt.ylabel('P(mu)*polarization')
    plt.legend(['Simulated','Fitted'])
    plt.show()
    """
    #YQ1 works!!!!!! move on to writing Q 
    
    def PQI_mu(x):
        return assocLegFit(x, cf2a, cf3a, cf4a, cf5a, cf6a, cf7a)
    
    
    pQI_mu = PQI_mu(mu)  
   # print("polarized intensity") 
    polarizedIntensity = h*vlya_z*n*pQI_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3)
   # print(polarizedIntensity)
    #for i in range(0,points,1):
    #    Qintensity_vobs_add = delta(vobs[i]-vlya_z,.001)*h*vobs[i]*n*pQI_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3) 
    #    Qintensity_vobs.append(Qintensity_vobs_add)
    
    """
    plt.plot(vobs,Qintensity_vobs)
    plt.xlim(-1+vlya_z, 1+vlya_z)
    plt.xlabel("frequency")
    plt.ylabel("Polarized Intensity(v)")
    plt.title("Polarized intensity %d %i %u" % (vi_index,T_index,nH_index))
    plt.show()
    """
    
    """
    plt.plot(data_direction,data_lastpos,".")
    plt.title("Position of Last Scatterign")
    plt.show()
    
    plt.plot(data_direction,data_frequency,".")
    plt.title("frequency")
    plt.show()
    
    plt.plot(data_direction,data_position,".")
    plt.title("Last Position")
    plt.show()
    """
    """
    ### For Extrap Hist
    
    frontLoc = findFrontLocation(cdataArray[:,2]) 
    print(frontLoc)
    
    fracMuExtrap = []
    tail=[]
    print("extreme extrap")
    countd=0
    for ind in range(0,len(data_i),1):
        #print("new")
        a= (data_vf[ind]-data_frequency[ind])/vlya 
        #print(a)
        b= H*(slabwidth*frontLoc - slabwidth*data_lastpos[ind])/(nH*(C*data_direction[ind] - u))
        #print(b)
        fracMu =a + b
        #print(fracMu)
        #fracMuExtrap.append(fracMu)
        if (np.abs(fracMu)>1):
            print(ind)
            print(fracMu)
            print("Denominator " + str(C*data_direction[ind] - u))
            tail.append(ind)
        else:
            if (countd<5):
                print(fracMu)
                print("Denominator " + str(C*data_direction[ind] - u))
                countd+=1
            fracMuExtrap.append(fracMu)
            
    sigmafracmu = np.std(fracMuExtrap)       
    print("STD is " + str(sigmafracmu))
    """
    

    """
    n, bins, patches = plt.hist(fracMuExtrap,1500)
    plt.title(str(T_index)+str(vi_index) +str(nH_index,))
    plt.xlim(-2.5*sigmafracmu,2.5*sigmafracmu)
    plt.show()        
    """            
    return (intensity, polarizedIntensity) #fracMuExtrap,tail)

def runsDataForErrors(ii,j,k,dataArray,cdataArray):
    
    print("started")
    
    with NumpyRNGContext(1):
        bootresult = bootstrap(dataArray, 1000)

    print("here")    

    bootstrapIntensities =[]
    bootstrapPolIntensities = []
    for w in range(0,len(bootresult),1):
        x,y = LyaObsFun1(ii,j,k,bootresult[w],cdataArray)
        bootstrapIntensities.append(x)
        bootstrapPolIntensities.append(y)
        
    """    
    plt.hist(bootstrapIntensities,30)
    plt.title("Intensities" + str(ii)+str(j)+str(k) + " n=1000")  
    plt.show()  
    plt.hist(bootstrapPolIntensities,30)
    plt.title("PolIntensities "+ str(ii)+str(j)+str(k) + " n=1000") 
    plt.show()
    """
    
    
    mean = np.mean(bootstrapIntensities)
    
    std=np.std(bootstrapIntensities)
    
    meanQ = np.mean(bootstrapPolIntensities)
    
    stdQ=np.std(bootstrapPolIntensities)
    #Intensities222 = np.array(bootstrapIntensities222)
    #PolIntensities222 = np.array(bootstrapPolIntensities222)
    return mean, meanQ, std, stdQ

directory = "/fs/project/PCON0003/yuanyuan/reionization/lastest_test/"
gridsize = 5
Intenlist = []
devIlist = []
PolIntensList =[]
devQList =[]
with open('EmilyOutputs2323M.txt', 'w') as f:
    f.write("Intensity \t Error \t PolarizedIntensity \t ErrorPol \n " )
    for i in range(3,5,1):
        for j in range(3,5,1):
            for k in range(0,5,1):
                run = str(i)+str(j)+str(k)
                if (run != "445"):
                    print(run)
                    scatterpath1 = directory +"lya_" + run+".txt"
                    #scatterpath2 = directory +"lya_" + run+"_2.txt"
                    temppath = directory +"output_" + str(i)+str(j)+".txt"
                    #if(run != '200' and run !="210"):
                    (dataArray,cdataArray)=  getFileDataArrays(i,j,k,scatterpath1,temppath) 
                    #intensity,polintensity,fracMu,tail = LyaObsFun1(j, i, k, dataArray, cdataArray)
                    mean,meanQ,std,stdQ = runsDataForErrors(i,j,k,dataArray,cdataArray)
                    print("intensity= " + str(mean))
                    print("std= " + str(std))
                    print("pol intensity= " + str(meanQ))
                    print()
                    print("stdPol= "+ str(stdQ))
                    f.write(str(mean)+ "\t" + str(std) + "\t" + str(meanQ) + "\t" + str(stdQ) + "\n")
                    Intenlist.append(mean)
                    PolIntensList.append(meanQ)
                    devIlist.append(std)
                    devQList.append(stdQ)
                   # nlist.append(x)
                   # if (run == "112"):
                       # taillist.append(y)    

f.close()


330
started
here
intensity= 1.0498227478452373e-13
std= 5.993429313575127e-16
pol intensity= 2.4606650874883644e-16

stdPol= 1.93472446032332e-16
331
started
here
intensity= 5.817591087100019e-13
std= 2.487782916377752e-15
pol intensity= 1.1791151345232734e-14

stdPol= 1.428839945162246e-15
332
started
here
intensity= 2.0243039995564002e-12
std= 6.973468853312361e-15
pol intensity= 6.404826445402595e-14

stdPol= 5.353710497916051e-15
333
started
here
intensity= 8.010839814450353e-12
std= 3.2144976842777134e-14
pol intensity= 2.2728523687835545e-13

stdPol= 2.773346189572498e-14
334
started
here
intensity= 2.1974501076164917e-11
std= 9.683919188785497e-14
pol intensity= 4.5434750028904323e-13

stdPol= 8.25346545070417e-14
340
started
here
intensity= 5.103466852800824e-13
std= 3.1252117571485915e-15
pol intensity= 2.747962780163984e-15

stdPol= 1.164974069936382e-15
341
started
here
intensity= 2.831533824420595e-12
std= 1.1615140924597003e-14
pol intensity= 9.330901393618289e-14

stdPol=

In [ ]:

import numpy as np
from astropy.stats import bootstrap
from astropy.utils import NumpyRNGContext
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def findFrontLocation(fracH):
    elementLocation = 0
    i=0
    while elementLocation == 0:
        if (fracH[i]-.5) > 0 :
            elementLocation=i
        i = i+1     
    return elementLocation  


def getFileDataArrays(i,j,k,scatter_path1,temperature_path):
    data = []
    data_i = []
    data_position = []
    data_direction = []
    data_polarization = []
    data_frequency = []
    data_freqoff = []
    data_lastpos = []
    data_vi=[]
    data_vf=[]
    
    
    with open(scatter_path1) as f:
        lines = f.readlines()[1:-1]
        for line in lines:
           data = line.split()
           for x in range(0, len(data), 9):
               data_i.append(int(data[x]))
               data_position.append(float(data[x+1]))
               data_direction.append(float(data[x+2]))
               data_polarization.append(float(data[x+3]))
               data_frequency.append(float(data[x+4]))
               data_freqoff.append(float(data[x+5]))
               data_lastpos.append(float(data[x+6]))
               data_vi.append(float(data[x+7]))
               data_vf.append(float(data[x+8]))
            
    
    
    dataArray = np.zeros((len(data_i),9))
    for i in range(0,len(data_i),1):
        dataArray[i,0]=data_i[i]
        dataArray[i,1]=data_position[i]
        dataArray[i,2]=data_direction[i]
        dataArray[i,3]=data_polarization[i]
        dataArray[i,4]=data_frequency[i]
        dataArray[i,5]=data_freqoff[i]
        dataArray[i,6]=data_lastpos[i]
        dataArray[i,7]=data_vi[i]
        dataArray[i,8]=data_vf[i]
        
       
    c_data = []
    c_j = []
    c_DNHImid = []
    c_y1H = []
    c_y1HE = []
    c_EH = []
    c_Te = []
    c_EHII = []
    c_THII = []
    c_EHI = []
    c_THI = []
    c_EHeI = []
    c_THeI = []
    c_EHeII = []
    c_THeII = []
    c_dEH = []      
    
    emptyline=0
    #here insert the path name to a file with the bottom output matrix from chenxiao's code 
    with open(temperature_path) as f:
        for _ in range(1):
            next(f)
        for line in f:
            if not line.strip():
                #print(emptyline)
                emptyline = emptyline+ 1
              
            if (emptyline ==4):
                next(f)
                next(f)
                next(f)
                next(f)
                emptyline  = emptyline+1
            if (emptyline ==5):
                c_data = line.split()
                for x in range(0, len(c_data), 15):
                    #print(line+ "here")
                    c_j.append(int(c_data[x]))
                    c_DNHImid.append(float(c_data[x+1]))
                    c_y1H.append(float(c_data[x+2]))
                    c_y1HE.append(float(c_data[x+3]))
                    c_EH.append(float(c_data[x+4]))
                    c_Te.append(float(c_data[x+5]))
                    c_EHII.append(float(c_data[x+6]))
                    c_THII.append(float(c_data[x+7]))
                    c_EHI.append(float(c_data[x+8]))
                    c_THI.append(float(c_data[x+9]))
                    c_EHeI.append(float(c_data[x+10]))
                    c_THeI.append(float(c_data[x+11]))
                    c_EHeII.append(float(c_data[x+12]))
                    c_THeII.append(float(c_data[x+13]))
                    c_dEH.append(float(c_data[x+14]))  
               
    cdataArray = np.zeros((len(c_j),15))
    for i in range(0,len(c_j),1):
        cdataArray[i,0]=c_j[i]
        cdataArray[i,1]=c_DNHImid[i]
        cdataArray[i,2]=c_y1H[i]
        cdataArray[i,3]=c_y1HE[i]
        cdataArray[i,4]=c_EH[i]
        cdataArray[i,5]=c_Te[i]
        cdataArray[i,6]=c_EHII[i]
        cdataArray[i,7]=c_THII[i]
        cdataArray[i,8]=c_EHI[i]
        cdataArray[i,9]=c_THI[i]
        cdataArray[i,10]=c_EHeI[i]
        cdataArray[i,11]=c_THeI[i]
        cdataArray[i,12]=c_EHeII[i]
        cdataArray[i,13]=c_THeII[i]
        cdataArray[i,14]=c_dEH[i]
    return dataArray, cdataArray

def give0(x,y):
    if (y==0):
        return 0
    else:
       return x/y 

def LyaObsFun1(vi_index,T_index,nH_index,data,c_data):
    h = 6.6261e-27 #erg s 
    points = 2000000
    vlya = 2.47e15   #in Hz   needs to be the uncorrected one 
    z=8
    vlya_z = vlya/(1+z)
    vobs = np.linspace(vlya_z -1,vlya_z + 1,points)   #make sure centered around redshifted vlya to current age
    theta= 3*np.pi/4  #will be adjusted later ADJUST THIS 
    mu= np.cos(theta) 
    intensity_vobs = []
    Qintensity_vobs = []
    RYD_K = 157800 #rydberg 
    #print("mu ")
    #print(mu)
    n=0  #initialize 
    nH = 1.37e-5 #hydrogen density at redshift    
    fhe = 0.079 # frac of he relative to hydrogen 
    slabwidth = 2.5e16 #FIX THIS
    
    H = 3.3149362e-17 # Hubble constant in the period of z = 8, in [s^-1]
    C= 29979245800 #speed of light, in [cm s^-1]
    
    
    
    
   # rangeR=(-.01,.01)
    if nH_index==1:
        nH=5e-5
       # rangeR=(-.01,.01)
    if nH_index==2:
        nH=1.37e-4
    if nH_index==3:
        nH=5e-4    
    if nH_index==4:
        nH=1.37e-3       
    u = 5e7
    if vi_index==1:
        u = 1e8
    if vi_index ==2:
        u=5e8
    if vi_index ==3:
        u=1e9
    if vi_index==4:
        u=5e9
    
    
    #data = []
    data_i = []
    data_position = []
    data_direction = []
    data_polarization = []
    data_frequency = []
    data_freqoff = []
    data_lastpos = []
    data_vi=[]
    data_vf=[]
    
    #the open function will be modified depend on your own directory
    #with open(scatter_path) as f:
    #    for _ in range(1):
    #        next(f)
    #    for line in f:
    #        data = line.split()
    #print(len(data))
    for x in range(0,len(data), 1):
        data_i.append(int(data[x,0]))
        data_position.append(float(data[x,1]))
        data_direction.append(float(data[x,2]))
        data_polarization.append(float(data[x,3]))
        data_frequency.append(float(data[x,4]))
        data_freqoff.append(float(data[x,5]))
        data_lastpos.append(float(data[x,6]))
        data_vi.append(float(data[x,7]))
        data_vf.append(float(data[x,8]))

         
    
    #c_data = []
    c_j = []
    c_DNHImid = []
    c_y1H = []
    c_y1HE = []
    c_EH = []
    c_Te = []
    c_EHII = []
    c_THII = []
    c_EHI = []
    c_THI = []
    c_EHeI = []
    c_THeI = []
    c_EHeII = []
    c_THeII = []
    c_dEH = []      
    
    #here insert the path name to a file with the bottom output matrix from chenxiao's code 

    for x in range(0, len(c_data), 15):
        c_j.append(int(c_data[x,0]))
        c_DNHImid.append(float(c_data[x,1]))
        c_y1H.append(float(c_data[x,2]))
        c_y1HE.append(float(c_data[x,3]))
        c_EH.append(float(c_data[x,4]))
        c_Te.append(float(c_data[x,5]))
        c_EHII.append(float(c_data[x,6]))
        c_THII.append(float(c_data[x,7]))
        c_EHI.append(float(c_data[x,8]))
        c_THI.append(float(c_data[x,9]))
        c_EHeI.append(float(c_data[x,10]))
        c_THeI.append(float(c_data[x,11]))
        c_EHeII.append(float(c_data[x,12]))
        c_THeII.append(float(c_data[x,13]))
        c_dEH.append(float(c_data[x,14]))
          
    def P0(x):
        return 1
    def P1(x):
        return x
    def P2(x):
        return (3*x**2)/2. - 1/2.
    def P3(x):
        return (5*x**3)/2. - 3*x/2.
    def P4(x):
        return (35*x**4)/8. - (15*x**2)/4. + 3/8.
    def P5(x):
        return (63*x**5)/8. - (35*x**3)/4. + 15*x/8.
    
    def delta(x,epsilon):
        return np.exp(-x*x/(4*epsilon))/(2*np.sqrt(np.pi * epsilon))
    
    def P22(x):
        return 3*(1-x**2)
    
    def P23(x):
        return 15*x*(1-x**2)
    
    def P24(x):
        return 15/2 * (7*x**2-1)*(1-x**2)
    
    def P25(x):
        return 1/4*(11*x*P24(x) - 7*P23(x))
    
    def P26(x):
        return 1/5*(13*x*P25(x) - 8*P24(x))
    
    def P27(x):
        return 1/6*(15*x*P26(x) - 9*P25(x))
    
    def assocLegFit(x,f2,f3,f4,f5,f6,f7):
        return f2*P22(x) + f3*P23(x) + f4*P24(x)+ f5*P25(x) +f6*P26(x)+f7*P27(x)
    
    
    
    mu_0 = []
    mu_1 = []
    mu_2 = []
    mu_3 = []
    mu_4 = []
    mu_5 = []
    
    for x in range(len(data_direction)):
        mu_0.append(P0(data_direction[x]))
        mu_1.append(P1(data_direction[x]))
        mu_2.append(P2(data_direction[x]))
        mu_3.append(P3(data_direction[x]))
        mu_4.append(P4(data_direction[x]))
        mu_5.append(P5(data_direction[x]))
        
    total_0 = sum(mu_0)
    total_1 = sum(mu_1)
    total_2 = sum(mu_2)
    total_3 = sum(mu_3)
    total_4 = sum(mu_4)
    total_5 = sum(mu_5)
    
    b0 = (2*0+1)*total_0/(2*len(data_direction))
    b1 = (2*1+1)*total_1/(2*len(data_direction))
    b2 = (2*2+1)*total_2/(2*len(data_direction))
    b3 = (2*3+1)*total_3/(2*len(data_direction))
    b4 = (2*4+1)*total_4/(2*len(data_direction))
    b5 = (2*5+1)*total_5/(2*len(data_direction))
    
    def P_mu(x):
        prob = (b0 + b1*P1(x) + b2*P2(x) + b3*P3(x) + b4*P4(x) + b5*P5(x))
        if (prob<0):
            return 0 
        else:
            return prob
    x1 = np.arange(-1, 1, 0.00001)
    y1 = np.arange(-1, 1, 0.00001)
    for l in range(0,len(x1),1):
        y1[l] = P_mu(x1[l])
    

    #print("b coeff") 
    #print(b0, b1, b2, b3, b4, b5)
    """
    hist,bin_edges, _ = plt.hist(data_direction, density=True,bins=1000)
    plt.plot(x1, y1)
    plt.legend(['Fitted', 'Simulated'])
    plt.title('Probability density of mu %d %i %u' % (T_index,vi_index,nH_index))
    plt.xlabel('Direction (mu)')
    plt.ylabel('Probability density (P(mu))')
    plt.show()
    """
    
    #photonspike = #send list to yuanyuan after meeting for 234 
    
    
    #freq_std = np.std(data_frequency)
    #in cm^3/s
    def qLya(Te):
        
        # 1s to 2p
        gammaL = 3.35e-1 + 1.45e-5*Te +1.39e-10*Te*Te - 5.66e-15*Te*Te*Te
        if(Te>2.5e4):
            gammaL = 3.28e-1 + 1.43e-5*Te -6.55e-12*Te*Te - 2.69e-18*Te*Te*Te
        q12p = 8.6287e-6/2./np.sqrt(Te)*gammaL*np.exp(-0.75*RYD_K/Te)
        
        # 1s to 3s
        gammaL = 8.27e-2 + 4.18e-6*Te -4.18e-10*Te*Te +9.37e-15*Te*Te*Te
        if (Te>2.5e4):
            gammaL= 7.31e-2 - 1.84e-7*Te +1.31e-12*Te*Te -1.62e-18*Te*Te*Te
        q13s = 8.6287e-6/2./np.sqrt(Te)*gammaL*np.exp(-0.8888888889*RYD_K/Te)
        
        # 1s to 3d
        gammaL = 1.21e-2 + 1.03e-5*Te -4.31e-10*Te*Te + 7.16e-15*Te*Te*Te
        if (Te>2.5e4):
            gammaL = 9.34e-2 + 1.03e-6*Te -3.68e-12*Te*Te +3.91e-18*Te*Te*Te
        q13d = 8.6287e-6/2./np.sqrt(Te)*gammaL*np.exp(-0.8888888889*RYD_K/Te)
        
        return (q12p + q13s + q13d)
        
    
    
    for j in range(0,len(c_j),1):
        n = n + slabwidth*nH*qLya(c_Te[j])*c_y1H[j]*(1-c_y1H[j] + fhe*(1-c_y1HE[j])) 
        #if (j%100 ==0):
         #   print(qLya(c_Te[j]))     
    
    p_mu = P_mu(mu)
    count1= 0
    ind=0
    """
    while (count1 == 0):
        if (bin_edges[ind]<mu and bin_edges[ind+1]>mu):
            count1 = count1+1
        ind = ind + 1  
        #print(ind)
    #print(bin_edges[ind-1])
    #print(bin_edges[ind])
    p_mu = hist[ind-1]
    #print('intensity')
    """
    intensity = h*vlya_z*n*p_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3)
   # print(intensity)
    #for i in range(0,points,1):
    #    intensity_vobs_add = delta(vobs[i]-vlya_z,.001)*h*vobs[i]*n*p_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3) 
    #    intensity_vobs.append(intensity_vobs_add)
    
    #delta(vobs[i]-vlya_z,.001)*
    """
    plt.plot(np.array(vobs),np.array(intensity_vobs))
    plt.xlim(-1+vlya_z, 1+vlya_z)
    plt.xlabel('frequency')
    plt.ylabel('Intensity(v)')
    plt.title('intensity curve as seen by observer %d %i %u' % (vi_index,T_index,nH_index) )
    plt.show()
    """
    
    #code to also give Q 
    
    
    # for Q, can we multiply the intesnsity by the polarization to give the directional affect 
    
    polar0 = []
    polar1 = []
    polar2 = []
    polar3 = []
    polar4 = []
    polar5 = []
    polar6 = []
    polar7 = []
    polar8 = []
    polar9 = []
    
    #vobsZ = np.linspace(np.min(data_frequency,max(data_frequency),points))
    
                        
    
    for x in range(len(data_direction)):
        if data_direction[x] < -0.8:
            polar0.append(data_polarization[x])
        if data_direction[x] >= -0.8 and data_direction[x] < -0.6:
            polar1.append(data_polarization[x])
        if data_direction[x] >= -0.6 and data_direction[x] < -0.4:
            polar2.append(data_polarization[x])
        if data_direction[x] >= -0.4 and data_direction[x] < -0.2:
            polar3.append(data_polarization[x])
        if data_direction[x] >= -0.2 and data_direction[x] < 0.0:
            polar4.append(data_polarization[x])
        if data_direction[x] >= 0.0 and data_direction[x] < 0.2:
            polar5.append(data_polarization[x])
        if data_direction[x] >= 0.2 and data_direction[x] < 0.4:
            polar6.append(data_polarization[x])
        if data_direction[x] >= 0.4 and data_direction[x] < 0.6:
            polar7.append(data_polarization[x])
        if data_direction[x] >= 0.6 and data_direction[x] < 0.8:
            polar8.append(data_polarization[x])
        if data_direction[x] >= 0.8 and data_direction[x] < 1.0:
            polar9.append(data_polarization[x])
   
    p0 = sum(polar0)/len(data_polarization)/0.2
    p1 = sum(polar1)/len(data_polarization)/0.2
    p2 = sum(polar2)/len(data_polarization)/0.2
    p3 = sum(polar3)/len(data_polarization)/0.2
    p4 = sum(polar4)/len(data_polarization)/0.2
    p5 = sum(polar5)/len(data_polarization)/0.2
    p6 = sum(polar6)/len(data_polarization)/0.2
    p7 = sum(polar7)/len(data_polarization)/0.2
    p8 = sum(polar8)/len(data_polarization)/0.2
    p9 = sum(polar9)/len(data_polarization)/0.2
    
    x2 = [-0.9, -0.7, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 0.7, 0.9]
    #y2 = [p0_avg, p1_avg, p2_avg, p3_avg, p4_avg, p5_avg, p6_avg, p7_avg, p8_avg, p9_avg]
    y3 = [p0, p1, p2, p3, p4, p5, p6, p7, p8, p9]
    """
    plt.plot(x2, y3)
    plt.title('Mean polarization in each direction bin')
    plt.xlabel('Direction (mu)')
    plt.ylabel('Probability density')
    plt.show()
    """
        
    #want to fit <q/i>P(mu) using associated lp, have polarization(mu) and Prob(mu)
    qIP2 = []
    qIP3 = []
    qIP4 = []
    qIP5 = []
    qIP6 = []   
    qIP7 = []
    histogram = np.zeros(len(data_direction))
    
    for x in range(len(data_direction)):
        histogram[x] = (data_direction[x]*data_polarization[x])
        qIP2.append(P22(data_direction[x]*data_polarization[x]))
        qIP3.append(P23(data_direction[x]*data_polarization[x]))
        qIP4.append(P24(data_direction[x]*data_polarization[x]))
        qIP5.append(P25(data_direction[x]*data_polarization[x]))
        qIP6.append(P26(data_direction[x]*data_polarization[x]))
        qIP7.append(P27(data_direction[x]*data_polarization[x]))
        
    
    totalq2 =sum(qIP2)
    totalq3 =sum(qIP3)
    totalq4 =sum(qIP4)
    totalq5 =sum(qIP5)
    totalq6 =sum(qIP6)
    totalq7 =sum(qIP7)
    
    totalQ = totalq2+totalq3+totalq4+totalq5+totalq6+totalq7 
    
    
    
    #popt, _ = curve_fit(assocLegFit, data_direction, histogram)
    #cf2,cf3,cf4,cf5,cf6,cf7 = popt
  
    #try again with weight of p in bin 
    
    q10 = []
    q11 = []
    q12 = []
    q13 = []
    q14 = []
    q15 = []
    q16 = []
    q17 = []
    q18 = []
    q19 = []
    
    
    for x in range(len(data_direction)):
        if data_direction[x] < -0.8:
            q10.append(P_mu(data_direction[x])*p0)
        if data_direction[x] >= -0.8 and data_direction[x] < -0.6:
            q11.append(P_mu(data_direction[x])*p1)
        if data_direction[x] >= -0.6 and data_direction[x] < -0.4:
            q12.append(P_mu(data_direction[x])*p2)
        if data_direction[x] >= -0.4 and data_direction[x] < -0.2:
            q13.append(P_mu(data_direction[x])*p3)
        if data_direction[x] >= -0.2 and data_direction[x] < 0.0:
            q14.append(P_mu(data_direction[x])*p4)
        if data_direction[x] >= 0.0 and data_direction[x] < 0.2:
            q15.append(P_mu(data_direction[x])*p5)
        if data_direction[x] >= 0.2 and data_direction[x] < 0.4:
            q16.append(P_mu(data_direction[x])*p6)
        if data_direction[x] >= 0.4 and data_direction[x] < 0.6:
            q17.append(P_mu(data_direction[x])*p7)
        if data_direction[x] >= 0.6 and data_direction[x] < 0.8:
            q18.append(P_mu(data_direction[x])*p8)
        if data_direction[x] >= 0.8 and data_direction[x] < 1.0:
            q19.append(P_mu(data_direction[x])*p9)
    
    
    w0 = give0(sum(q10),len(q10))
    w1 = give0(sum(q11),len(q11))
    w2 = give0(sum(q12),len(q12))
    w3 = give0(sum(q13),len(q13))
    w4 = give0(sum(q14),len(q14))
    w5 = give0(sum(q15),len(q15))
    w6 = give0(sum(q16),len(q16))
    w7 = give0(sum(q17),len(q17))
    w8 =give0(sum(q18),len(q18))
    w9 = give0(sum(q19),len(q19))
    
    yw = [w0,w1,w2,w3,w4,w5,w6,w7,w8,w9]
    ywarray = np.array(yw)
    x2array = np.array(x2)
    x1array = np.array(x1)
    
    popt, _ = curve_fit(assocLegFit, x2array, ywarray)
    cf2a,cf3a,cf4a,cf5a,cf6a, cf7a = popt
    
    yQ1 = cf2a*P22(x1array) + cf3a*P23(x1array) + cf4a*P24(x1array)+ cf5a*P25(x1array) + cf6a*P26(x1array) + cf7a*P27(x1array)
    yQ2 = cf2a*P22(x2array) + cf3a*P23(x2array) + cf4a*P24(x2array)+ cf5a*P25(x2array) + cf6a*P26(x2array) + cf7a*P27(x2array)
    """
    plt.hist(histogram, density=True)
    plt.title('Probability density of mu*polarization')
    plt.xlabel('Direction (mu)')
    plt.ylabel('<Q/I>P(mu)')
    plt.show()
    
    #plt.plot(x2,yw)
    plt.plot(x2,yQ2)
    plt.plot(x1,yQ1)
    plt.title('P(mu)*polarization')
    plt.xlabel('Direction (mu)')
    plt.ylabel('P(mu)*polarization')
    plt.legend(['Simulated','Fitted'])
    plt.show()
    """
    #YQ1 works!!!!!! move on to writing Q 
    
    def PQI_mu(x):
        return assocLegFit(x, cf2a, cf3a, cf4a, cf5a, cf6a, cf7a)
    
    
    pQI_mu = PQI_mu(mu)  
   # print("polarized intensity") 
    polarizedIntensity = h*vlya_z*n*pQI_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3)
   # print(polarizedIntensity)
    #for i in range(0,points,1):
    #    Qintensity_vobs_add = delta(vobs[i]-vlya_z,.001)*h*vobs[i]*n*pQI_mu/(2*np.pi*np.abs(np.cos(theta))*(1+z)**3) 
    #    Qintensity_vobs.append(Qintensity_vobs_add)
    
    """
    plt.plot(vobs,Qintensity_vobs)
    plt.xlim(-1+vlya_z, 1+vlya_z)
    plt.xlabel("frequency")
    plt.ylabel("Polarized Intensity(v)")
    plt.title("Polarized intensity %d %i %u" % (vi_index,T_index,nH_index))
    plt.show()
    """
    
    """
    plt.plot(data_direction,data_lastpos,".")
    plt.title("Position of Last Scatterign")
    plt.show()
    
    plt.plot(data_direction,data_frequency,".")
    plt.title("frequency")
    plt.show()
    
    plt.plot(data_direction,data_position,".")
    plt.title("Last Position")
    plt.show()
    """
    """
    ### For Extrap Hist
    
    frontLoc = findFrontLocation(cdataArray[:,2]) 
    print(frontLoc)
    
    fracMuExtrap = []
    tail=[]
    print("extreme extrap")
    countd=0
    for ind in range(0,len(data_i),1):
        #print("new")
        a= (data_vf[ind]-data_frequency[ind])/vlya 
        #print(a)
        b= H*(slabwidth*frontLoc - slabwidth*data_lastpos[ind])/(nH*(C*data_direction[ind] - u))
        #print(b)
        fracMu =a + b
        #print(fracMu)
        #fracMuExtrap.append(fracMu)
        if (np.abs(fracMu)>1):
            print(ind)
            print(fracMu)
            print("Denominator " + str(C*data_direction[ind] - u))
            tail.append(ind)
        else:
            if (countd<5):
                print(fracMu)
                print("Denominator " + str(C*data_direction[ind] - u))
                countd+=1
            fracMuExtrap.append(fracMu)
            
    sigmafracmu = np.std(fracMuExtrap)       
    print("STD is " + str(sigmafracmu))
    """
    

    """
    n, bins, patches = plt.hist(fracMuExtrap,1500)
    plt.title(str(T_index)+str(vi_index) +str(nH_index,))
    plt.xlim(-2.5*sigmafracmu,2.5*sigmafracmu)
    plt.show()        
    """            
    return (intensity, polarizedIntensity) #fracMuExtrap,tail)

def runsDataForErrors(ii,j,k,dataArray,cdataArray):
    
    print("started")
    
    with NumpyRNGContext(1):
        bootresult = bootstrap(dataArray, 1000)

    print("here")    

    bootstrapIntensities =[]
    bootstrapPolIntensities = []
    for w in range(0,len(bootresult),1):
        x,y = LyaObsFun1(ii,j,k,bootresult[w],cdataArray)
        bootstrapIntensities.append(x)
        bootstrapPolIntensities.append(y)
        
    """    
    plt.hist(bootstrapIntensities,30)
    plt.title("Intensities" + str(ii)+str(j)+str(k) + " n=1000")  
    plt.show()  
    plt.hist(bootstrapPolIntensities,30)
    plt.title("PolIntensities "+ str(ii)+str(j)+str(k) + " n=1000") 
    plt.show()
    """
    
    
    mean = np.mean(bootstrapIntensities)
    
    std=np.std(bootstrapIntensities)
    
    meanQ = np.mean(bootstrapPolIntensities)
    
    stdQ=np.std(bootstrapPolIntensities)
    #Intensities222 = np.array(bootstrapIntensities222)
    #PolIntensities222 = np.array(bootstrapPolIntensities222)
    return mean, meanQ, std, stdQ

directory = "/fs/project/PCON0003/yuanyuan/reionization/lastest_test/"
gridsize = 5
Intenlist = []
devIlist = []
PolIntensList =[]
devQList =[]
with open('EmilyOutputs2323_A.txt', 'w') as f:
    f.write("Intensity \t Error \t PolarizedIntensity \t ErrorPol \n " )
    for i in range(3,5,1):
        for j in range(0,5,1):
            for k in range(0,5,1):
                run = str(i)+str(j)+str(k)
                if (run != "445"):
                    print(run)
                    scatterpath1 = directory +"lya_" + run+".txt"
                    #scatterpath2 = directory +"lya_" + run+"_2.txt"
                    temppath = directory +"output_" + str(i)+str(j)+".txt"
                    #if(run != '200' and run !="210"):
                    (dataArray,cdataArray)=  getFileDataArrays(i,j,k,scatterpath1,temppath) 
                    #intensity,polintensity,fracMu,tail = LyaObsFun1(j, i, k, dataArray, cdataArray)
                    mean,meanQ,std,stdQ = runsDataForErrors(i,j,k,dataArray,cdataArray)
                    print("intensity= " + str(mean))
                    print("std= " + str(std))
                    print("pol intensity= " + str(meanQ))
                    print()
                    print("stdPol= "+ str(stdQ))
                    f.write(str(mean)+ "\t" + str(std) + "\t" + str(meanQ) + "\t" + str(stdQ) + "\n")
                    Intenlist.append(mean)
                    PolIntensList.append(meanQ)
                    devIlist.append(std)
                    devQList.append(stdQ)
                   # nlist.append(x)
                   # if (run == "112"):
                       # taillist.append(y)    

f.close()


300
started
here
